<a href="https://colab.research.google.com/github/DarkLord-13/Machine-Learning-01/blob/main/Blog_Writer_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q crewai
!pip install -q 'crewai[tools]'
!pip install -q langchain_groq
!pip install -q crewai groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:

In [ ]:
import os
from getpass import getpass

from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool
from langchain_groq import ChatGroq
from crewai.tools import tool

In [ ]:
SERPER_API_KEY = "" # getpass("Your serper api key: ")
os.environ['SERPER_API_KEY'] = ''

GROQ_API_KEY = "" # getpass("Your Groq api key: ")
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [ ]:
llm = LLM(model="groq/gemma2-9b-it", api_key="")

In [ ]:
llm.call('hi')

'Hi there! 👋 How can I help you today?\n'

In [ ]:
search_tool = SerperDevTool()

In [ ]:
researcher = Agent(
    role = "Researcher",
    goal='Pioneer revolutionary advancements in {subject_area}',
    backstory=(
    "your research is used by people of all fields to make key decisions."
    "your research done on the topic is always factual, highly detailed and listed out in points."
    "you always ensure to mention all the involved parties in current topic and mention their contribution positively or negatively."
    "your research is unbiased and to the point."
    ),
    llm = llm,
    max_iter = 3,
    tools = [search_tool],
    allow_delegation=True,
    verbose=True
)

writer = Agent(
    role = "Writer",
    goal='Craft direct and factual narratives about {subject_area}',
    verbose=True,
    backstory=(
    "you are a skilled writer that specilizes in writing the most factual stories."
    "you are known for your accuarate, detailed, factual and direct writing skill."
    "you always ensure to mention all the adversaries involved in a topic and write their involvements and contribution to the topic in detail."
    ),
    tools = [search_tool],
    llm = llm,
    max_iter = 3,
    allow_delegation=False
)

In [ ]:
research_task = Task(
    description = (
    "Explore and identify the current development within {subject_area}"
    "Detailed SEO report of the development in a comprehensive narrative."
  ),
    expected_output='A report, structured into three detailed paragraphs',
    tools = [search_tool],
    agent = researcher
)

write_task = Task(
    description=(
    "Craft an engaging article on recent advancements  within {subject_area}"
    "The article should be clear, detailed, factual, with sources mentioned, listed in points."
  ),
    expected_output='A blog article on current update about {subject_area} in markdown.',
    tools = [search_tool],
    agent = writer,
    async_execution = False,
    output_file = "blog.md"
)

In [ ]:
crew = Crew(
    agents = [researcher, writer],
    tasks = [research_task, write_task],
    process = Process.sequential,
    max_rpm = 1,
    cache=True
)

inputs = {'subject_area': 'New Delhi Elections 2025'}
result = crew.kickoff(inputs=inputs)

print(result)

# Agent: Researcher
## Task: Explore and identify the current development within New Delhi Elections 2025Detailed SEO report of the development in a comprehensive narrative.


# Agent: Researcher
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"New Delhi Elections 2025 developments\"}"
## Tool Output: 
{'searchParameters': {'q': 'New Delhi Elections 2025 developments', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '2025 Delhi Legislative Assembly election - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/2025_Delhi_Legislative_Assembly_election', 'snippet': 'The 2025 Delhi Legislative Assembly elections were held in Delhi on 5 February 2025 to elect all 70 members of the Delhi Legislative Assembly.', 'position': 1, 'sitelinks': [{'title': 'Elections in Delhi', 'link': 'https://en.wikipedia.org/wiki/Elections_in_Delhi'}, {'title': 'New Delhi Assembly...', 'link': 'https://en.wikipedia.org/wiki/New_Delhi_Assembly_constituen